In [47]:
# Import packages
import pandas as pd
from google.cloud import bigquery
from google.cloud import bigquery_storage
import warnings
warnings.filterwarnings(action="ignore")

In [34]:
# Pull the data from bigquery

# Instantiate a BigQuery client
client = bigquery.Client(project="dh-logistics-product-ops")
bqstorage_client = bigquery_storage.BigQueryReadClient()

# Declare the query
query = """
    SELECT
        region,
        entity_id,
        asa_id,
        asa_name,
        master_asa_id,
        asa_common_name,
        vendor_code,
        is_lb_lm,
        update_timestamp,
        DATE(update_timestamp) AS update_date
    FROM `dh-logistics-product-ops.pricing.final_vendor_list_all_data_loved_brands_scaled_code`
    WHERE DATE(update_timestamp) >= DATE("2023-03-01")
"""

df_vendors = client.query(query).result().to_dataframe(bqstorage_client=bqstorage_client, progress_bar_type="tqdm")

Downloading: 100%|██████████| 11060430/11060430 [01:35<00:00, 115352.26rows/s]


In [96]:
# Create a data frame that contains the data from the current pipeline run
df_lbs_current_timestamp = df_vendors[df_vendors["update_timestamp"] == df_vendors["update_timestamp"].max()]

# Create a list with all the iterables
iter_list = df_vendors[df_vendors["update_timestamp"] < df_vendors["update_timestamp"].max()][["entity_id", "master_asa_id", "update_timestamp"]]\
    .drop_duplicates()\
    .reset_index(drop=True)\
    .sort_values(["entity_id", "master_asa_id", "update_timestamp"])\
    .values.tolist()

# Iterate over all entity_ids and master_asa_ids and calculate the match percentage
match_percentage_list = []
for idx, i in enumerate(iter_list): # Make sure to remove [0:20] when you implement the parallelization
    # Print a status message
    print(f"This is iteration {idx} out of {len(iter_list)} --> {i}")

    # Create a data frame that contains the full information about a specific entity_id, master_asa_id, and a PREVIOUS update_timestamp
    df_iter_previous = df_vendors[(df_vendors["entity_id"] == i[0]) & (df_vendors["master_asa_id"] == i[1]) & (df_vendors["update_timestamp"] == i[2])].reset_index(drop=True)
    
    # Create a data frame that contains the full information about a specific entity_id, master_asa_id, and the CURRENT update_timestamp
    df_iter_current = df_lbs_current_timestamp[(df_lbs_current_timestamp["entity_id"] == i[0]) & (df_lbs_current_timestamp["master_asa_id"] == i[1])].reset_index(drop=True)
    
    # Extract the LBs and NLBs from this data frame and place them in a list
    lb_iter = df_iter_previous[df_iter_previous["is_lb_lm"] == "Y"]["vendor_code"].tolist()
    nlb_iter = df_iter_previous[df_iter_previous["is_lb_lm"] == "N"]["vendor_code"].tolist()

    # Extract the LBs and NLBs from df_lbs_current_timestamp and place them in a list
    lb_current = df_iter_current[df_iter_current["is_lb_lm"] == "Y"]["vendor_code"].tolist()
    nlb_current = df_iter_current[df_iter_current["is_lb_lm"] == "N"]["vendor_code"].tolist()

    # Use the difflib library to compute the match percentage between lb_iter and lb_current as well as nlb_iter and nlb_current
    try:
        match_percentage_lb = round(len(list(set(lb_iter) & set(lb_current))) / len(lb_current), 4)
    except ZeroDivisionError: # If there are no *LBs* at all in the current list, then the division would yield an error. This try-except block handles this
        match_percentage_lb = None
    
    try:
        match_percentage_nlb = round(len(list(set(nlb_iter) & set(nlb_current))) / len(nlb_current), 4)
    except ZeroDivisionError: # If there are no *NLBs* at all in the current list, then the division would yield an error. This try-except block handles this
        match_percentage_nlb = None

    # Create the output_dict
    output_dict = {
        "entity_id": i[0],
        "master_asa_id": i[1],
        "update_timestamp": i[2],
        "update_date": pd.to_datetime(i[2]).date(),
        "match_percentage_lb": match_percentage_lb,
        "match_percentage_nlb": match_percentage_nlb,
    }

    # Append the output_dict to match_percentage_list
    match_percentage_list.append(output_dict)

# Convert match_percentage_list to a data frame
df_match_percentage = pd.DataFrame(match_percentage_list)
    

This is iteration 0 out of 77869 --> ['AP_PA', 4, Timestamp('2023-03-03 18:02:25.506850+0000', tz='UTC')]
This is iteration 1 out of 77869 --> ['AP_PA', 4, Timestamp('2023-03-09 20:26:13.610055+0000', tz='UTC')]
This is iteration 2 out of 77869 --> ['AP_PA', 4, Timestamp('2023-03-16 20:10:40.583637+0000', tz='UTC')]
This is iteration 3 out of 77869 --> ['AP_PA', 4, Timestamp('2023-03-24 15:30:27.938936+0000', tz='UTC')]
This is iteration 4 out of 77869 --> ['AP_PA', 4, Timestamp('2023-04-01 07:20:01.005647+0000', tz='UTC')]
This is iteration 5 out of 77869 --> ['AP_PA', 4, Timestamp('2023-04-06 19:52:02.738808+0000', tz='UTC')]
This is iteration 6 out of 77869 --> ['AP_PA', 4, Timestamp('2023-04-13 20:21:40.107578+0000', tz='UTC')]
This is iteration 7 out of 77869 --> ['AP_PA', 4, Timestamp('2023-04-20 19:55:50.615858+0000', tz='UTC')]
This is iteration 8 out of 77869 --> ['AP_PA', 4, Timestamp('2023-04-27 20:14:02.849357+0000', tz='UTC')]
This is iteration 9 out of 77869 --> ['AP_PA',